# futureEXPERT usage reference: Forecasting with covariates

## Log in to the _futureEXPERT_ client
_futureEXPERT_ needs credentials to authenticate you. So, to log in, you need to create an authenticated client which handles all interactions with the _future_ platform.

If you do not yet have an account for the _future_ platform, you can apply for a trial account [here](https://www.prognostica.de/de/future.html).

Additionally, if you are not subscribed for futureEXPERT, please contact us at support@future.prognostica.de

If you are using futureEXPERT for the first time, we recommend that you begin with [Getting Started](simple_workflow.ipynb)

In [ ]:
from futureexpert import ExpertClient

# You can provide your credentials from an .env file (see .env.example)
import dotenv
dotenv.load_dotenv()

# Log in using your credentials or alternatively provide FUTURE_USER and FUTURE_PW via environment variables or .env file
client = ExpertClient(user='', password='')

### Configure and run forecasting with covariates

In some cases, it is advantageous to include additional data as influencing variables in the model. In the example of bicycle data on a daily basis, for demonstration purposes, we show the inclustion of only one covariate &ndash; working days &ndash; although holidays and weather data is increasing the forecast quality as well.

In order to generate a forecast which takes covariates into account, create the covariates first. Define a data definition and a time series creation config similar as for the forecasting data.

The returned identification number must then be added to the report configuration.

You can use the [futureMATCHER](notebooks/cov_matcher_and_forecast.ipynb) to calculate the best lags for each covariate and create a ranking of each covariate based on their predictive power.

In [ ]:
from futureexpert import *

# Upload Working Days to use them as covariate
covs_version = client.check_in_time_series(raw_data_source='../example-data/working_days_bavaria.csv',
                                          data_definition=DataDefinition(date_columns=DateColumn(name='time', format='%Y-%m-%d'),
                                                                         value_columns=[ValueColumn(name='value')]),
                                          config_ts_creation=TsCreationConfig(time_granularity='daily',
                                                                              value_columns_to_save=['value']),
                                          file_specification=FileSpecification(delimiter=';', decimal=','))

# Create the configuaration for the data you want to forecast
data_definition = DataDefinition(date_columns=DateColumn(name='date', format='%Y-%m-%d'),
                                 value_columns=[ValueColumn(name='value')])

config_2 = TsCreationConfig(time_granularity='daily', value_columns_to_save=['value'])

fc_report_config = ReportConfig(title='My first report with covariate',
                                covs_version=covs_version, covs_lag=0,  # add
                                forecasting=ForecastingConfig(fc_horizon=3,
                                                              confidence_level=0.95),
                                preprocessing=PreprocessingConfig(use_season_detection=True,
                                                                  detect_outliers=True,
                                                                  replace_outliers=True)
                                )

forecast_identifier = client.create_forecast_from_raw_data(raw_data_source='../example-data/bicycle_data_single.csv',
                                                           data_definition=data_definition,
                                                           config_ts_creation=config_2,
                                                           config_fc=fc_report_config)
client.get_report_status(forecast_identifier).print()